In [9]:
import numpy as np
from scipy.spatial import distance
import sys
print sys.executable
print sys.path
import collections
import re
#from spacy.lang.en.stop_words import STOP_WORDS
#from gensim.models.phrases import Phrases, Phraser
#from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import glob

#import tensorflow as tf
import tensorflow_hub as hub
#import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from scipy.spatial import distance

#to write all essays into one text file 0.txt
'''with open('0.txt', 'w') as outfile:
    for i in range(1,22):
        with open('./server/data/Essays/'+str(i)+'.txt') as infile:
            for line in infile:
                outfile.write(line)'''

#import seaborn as sns
#import nltk
#nltk.download('punkt')

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/4","https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

/home/shadek/anaconda2/envs/tfhub/bin/python
['', '/home/shadek/anaconda2/envs/tfhub/lib/python27.zip', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/plat-linux2', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/lib-tk', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/lib-old', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/lib-dynload', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/site-packages', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/site-packages', '/home/shadek/anaconda2/envs/tfhub/lib/python2.7/site-packages/IPython/extensions', '/home/shadek/.ipython']
module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [10]:
#to check against keywords in trained model
candidate_words = ['curiosity', 'learning', 'root_cause', 'encourage', 'encouraging', 'questioning', 'question_asked', 
                   'afraid', 'daily', 'daily_basis', 'conversation', 'conversational', 'open', 'communication', 'positive', 'positive_feedback', 'positive_response',
                  'environment', 'inviting', 'participation', 'confidence', 'confidencebuilding', 'orientation'
                  ,'expectation', 'equally', 'valuable', 'mistake' , 'example', 'share_common', 'share_similar', 'sharealike', 'fear',
                  'admit', 'admitting', 'accountability', 'insecurity', 'speak','provider', 'anxiety', 'gain','culture',
                  'retaliation','role_model', 'demonstrated','leader', 'guilt', 'guilty', 'leadership', 'support', 'rounding',
                  'visible', 'relatable', 'personal', 'relationship', 'policy', 'demonstrate']

candidate_sentences = [
                        'everyone\'s voice and input is valuable and needed.',
                       'feel that they are just as important as everybody around', 
                        'open up daily conversations.',
                        'more than one person was also questioning this',
                        'everybody is important',
                        'mutual respect',
                        'communicate with teammates',
                        'Encourage questioning', 'greater participation',
                        'must be confident',
    
                        'encourage people to speak up',
                        'reduce the anxiety and make them excited to speak',
                        'more willing to discuss them because they had psychological safety',
                        'encouraged to speak up',
                        'working on a learning problem',
                        'modeling curiosity',
    
                        'admitting our mistakes', 'bring down everyone\'s anxiety at work',
                        'motivation and accountability', 'can feel confident as well', 'can make mistakes',
                        'point out the mistakes', 
    
    
                        'creating an inviting environment',
                        'not be shamed',
    
    
                        'start a conversation and engage them',
                        'people become comfortable to speak up when they feel heard and invited to speak up',
                        'important that this is in place so that errors are brought up and discussed and not hid.',
                        'should be empowered by leadership', 'creating an environment that draws people',
                        'feel safe in order for them to speak up', ' feel safe about discussing',
                        'feel unsafe to speak up', 'let management know and we will help',
                        'Management should seek opinions of the staff',
                        'takes more ownership of her position and has pride in the job'                   
                      ]
STOP_WORDS  = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

#filename='./server/data/wiki_4gram_50d.txt'
filename='./../../../PhraseModel/wiki_4gram_50d.txt'
lemmatizer = WordNetLemmatizer()

In [11]:

# simple Least Recently Used Cache
# inspired by https://www.kunxi.org/blog/2014/05/lru-cache-in-python/
class LRUCache:
  def __init__(self, capacity):
    self._cache = collections.OrderedDict()
    self._capacity = capacity

  def __setitem__(self, key, value):
    if key in self._cache:
      self._cache.pop(key)
    elif len(self._cache) >= self._capacity:
      self._cache.popitem(last=False)
    self._cache[key] = value

  def __getitem__(self, key):
    if key in self._cache:
      value = self._cache.pop(key)
      # put in the back
      self._cache[key] = value
      return value
    else:
      return None

  def __contains__(self, key):
    return key in self._cache

In [12]:
def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9_\s]', '', sentence)
    return re.sub(r'\s{2,}', ' ', sentence)
def tokenize(sentence):
    return [lemmatizer.lemmatize(token) for token in sentence.split() if ((token not in STOP_WORDS)
                                                                        and (not token.isdigit()))]

In [13]:
header_read = False
skip_head=True
#dist_type = 'cosine' #'euclidean'
vocabulary = []
dictionary = {}
embeddings = None
cache_capacity=10000

with open(filename, 'r') as filehandler:
  numbers = []
  for line in filehandler:
    if skip_head and not header_read:
      header_read = True
      pass
    else:
      split = line.split(' ')
      dictionary[split[0]] = len(vocabulary)
      vocabulary.append(split[0])
      numbers.append([float(x) for x in split[1:]]) #split[1:]

embeddings = np.array(numbers, dtype=np.float32)
_cache = LRUCache(cache_capacity)

def get_embedding_for_a_word( word):
    return get_embedding_for_words([word])

def get_word( index):
    if index < len(vocabulary):
      return vocabulary[index]
    else:
      return None

def get_embedding_for_words(words):
    indicies = [dictionary[word] for word in words if word in dictionary]
    if len(indicies) > 0:
      embedding = np.mean(embeddings[indicies, :], axis=0)
    else:
      embedding = np.zeros(embeddings.shape[1],)
    return embedding

def get_sentence_array(essayname, candidate_sent):
    essaystr = ''
    #candidate_words = word_tokenize(' '.join(tokenize(clean_sentence(candidate_sent))))
    #num_tokens = len([word for word in candidate_words if word in dictionary]) #check how many tokens are actually in phrase model vocab
    #a,b =  max(2, num_tokens/2), num_tokens # min and max size of n-gram tokens based on candidate sentence token size
    with open('./server/data/Essays/'+essayname, 'r') as filehandler:
        for line in filehandler:
            essaystr = essaystr + ' ' + line
    essaystr = essaystr.replace("\r","")
    essaystr = essaystr.replace("\n","")
    sentences = sent_tokenize(essaystr)
    ngrams = []
    for sentence in sentences:
        ngrams.append(sentence)    
    return ngrams

def get_embeddings_for_an_essay(essayname):
    tokens = set()
    with open('./server/data/Essays/'+essayname, 'r') as filehandler:
        for line in filehandler:
            cleaned_sentence = clean_sentence(line)
            tokenized_sentence = tokenize(cleaned_sentence)
            for t in tokenized_sentence:
                tokens.add(t)
    indicies = [dictionary[word] for word in tokens if word in dictionary]
    token_embeddings = embeddings[indicies, :]
    return indicies, token_embeddings
            
def compute_all_distances_from_a_word(word, essayname, dist_type):
    cache_value = _cache[word]
    if word in _cache:
      #print 'cache hit:', word
      return _cache[word]
    #print 'cache miss:', word
    embedding = get_embedding_for_a_word(word)
    embedding = embedding.reshape(1, embeddings.shape[1])
    token_indicies, essay_token_embeddings = get_embeddings_for_an_essay(essayname)
    dists = distance.cdist(embedding, essay_token_embeddings, dist_type)[0,:]
    _cache[word] = dists
    #print token_indicies, dists
    return token_indicies, dists

def recommend_words_by_avg_dist(word, essayname, how_many=10, dist_type='euclidean' ):
    token_indicies, dists = compute_all_distances_from_a_word(word, essayname, dist_type)
    res = sorted(list(zip(token_indicies, dists)), key = lambda x: x[1])
    return [(vocabulary[index], "{:.2f}".format(dist)) for index, dist in res[:how_many]]

# phrase2Vec model experiments 

In [26]:
def embed(input):
  return model(input)

distance_threshold = 0.7

themesentencefiles = glob.glob('./../../../PhraseModel/Themes/4gram*.txt')
for i, file in enumerate(themesentencefiles):
    print file
    themefileoriginal = file[file.rfind("/") + 6:]
    print themefileoriginal
    f = open('./../../../PhraseModel/Themes/'+ themefileoriginal)
    themelines = f.readlines()
    with open(file, 'r') as filehandler:
        themevectors = []
        use_themevectors = [] #universal sentence encoder embedding
        for fileindex, line in enumerate(filehandler):
            tokenized_sentence = tokenize(line)
            theme_sentence_embedding = get_embedding_for_words(tokenized_sentence)
            use_embedding = embed([themelines[fileindex]])
            themevectors.append((line, theme_sentence_embedding, themelines[fileindex]))
            use_themevectors.append((line, use_embedding[:1], themelines[fileindex]))
    essayfiles = glob.glob('./../../../PhraseModel/StudentEssay/M2V3/4gram*.txt')
    for j, essay in enumerate(essayfiles):
        
        print "---" + essay
        essayfileoriginal = essay[essay.rfind("/") + 6:]
        f = open('./../../../PhraseModel/StudentEssay/M2V3/'+ essayfileoriginal)
        essaylines = f.readlines()
        with open(essay, 'r') as essayhandler:
            essayvectors = []
            use_essayvectors = []
            for fileindex, line in enumerate(essayhandler):
                tokenized_sentence = tokenize(line)
                essay_sentence_embedding = get_embedding_for_words(tokenized_sentence)
                encoder_embedding = embed([essaylines[fileindex]])
                essayvectors.append((line, essay_sentence_embedding, essaylines[fileindex]))
                use_essayvectors.append((line, encoder_embedding[:1], essaylines[fileindex]))
            closest_theme, closest_sentence = '',''
            min_dist = float('inf')
            use_closest_theme, use_closest_sentence = '',''
            use_min_dist = float('inf')
            mn,mx = float('inf'), float('-inf')
            for index1, t in enumerate(themevectors):
                if index1 > 0:
                    break
                for index2, e in enumerate(essayvectors):
                    #print t[1], e[1]
                    dists = distance.cdist([t[1]], [e[1]], 'cosine')[0,:]
                    if dists[0] < min_dist:
                        closest_theme, closest_sentence = t[2], e[2] #t[0], e[0]
                        min_dist = dists[0]
                    mn = min(mn, dists[0])
                    mx = max(mx, dists[0])
                    use_dists = distance.cdist(use_themevectors[index1][1], use_essayvectors[index2][1], 'cosine')[0,:]
                    if use_dists[0] < use_min_dist:
                        use_closest_theme, use_closest_sentence = use_themevectors[index1][2], use_essayvectors[index2][2]
                        use_min_dist = use_dists[0]
                    
            print "{:.2f}".format(min_dist), closest_theme, closest_sentence
            print '-----'
            print "{:.2f}".format(use_min_dist), use_closest_theme, use_closest_sentence
            #print mn, mx

./../../../PhraseModel/Themes/4gram_administrative_leadership.txt
_administrative_leadership.txt
---./../../../PhraseModel/StudentEssay/M2V3/4gram_101.txt
0.36 Do they view me as approachable and open to discussion?
The surgeon might feel ganged up on but we explain in a calm manner that it is not a critique of their practise but for our education.

-----
0.80 Do they view me as approachable and open to discussion?
The down side is when I tell nurses to go and ask the surgeon, the direct their displeasure at me.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_93.txt
0.27 Do they view me as approachable and open to discussion?
For these individuals, the wellness plan may encompass the following: 1) Talking to an active listener.

-----
0.84 Do they view me as approachable and open to discussion?
For these individuals, the wellness plan may encompass the following: 1) Talking to an active listener.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_20.txt
0.34 Do they view me as appro

0.37 Do they view me as approachable and open to discussion?
Everyone is entitled to an opinion and their feelings.

-----
0.71 Do they view me as approachable and open to discussion?
I do not believe anyone can learn when they feel intimidated to ask questions.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_92.txt
0.31 Do they view me as approachable and open to discussion?
People need to feel safe and secure, comfortable in speaking up and knowing that if they do speak up they will not be ostracized or viewed as lacking knowledge.

-----
0.73 Do they view me as approachable and open to discussion?
People need to feel safe and secure, comfortable in speaking up and knowing that if they do speak up they will not be ostracized or viewed as lacking knowledge.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_50.txt
0.29 Do they view me as approachable and open to discussion?
It starts with a culture of admission.

-----
0.83 Do they view me as approachable and open to discussion?
We

0.32 Do they view me as approachable and open to discussion?
I enjoy coming to work and actually look forward to it.

-----
0.77 Do they view me as approachable and open to discussion?
Let them know that is it okay to ask questions and to not be afraid to do so.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_96.txt
0.44 Do they view me as approachable and open to discussion?
I tell them that it is their patient as well, and we need to work together to provide good patient care.

-----
0.72 Do they view me as approachable and open to discussion?
I tell them that it is their patient as well, and we need to work together to provide good patient care.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_12.txt
0.22 Do they view me as approachable and open to discussion?
Once the meeting begins, we sit in a circle, taking turns discussing concerns.

-----
0.71 Do they view me as approachable and open to discussion?
I am frequently asked to attend as many unit council meetings because I ha

0.32 In step 2, I would admit to my own fallibility.
I won’t go into the details, but I have never been able to get past this error and I left bedside nursing in the ICU because of it.

-----
0.78 In step 2, I would admit to my own fallibility.
Now I am silent unless someone asks to hear from me by name.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_88.txt
0.26 In step 2, I would admit to my own fallibility.
In nursing school when a question was asked, and I felt I knew the answer, I still would not raise my hand to be called on.

-----
0.72 In step 2, I would admit to my own fallibility.
I have a personal battle within myself when it comes to speaking up at work.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_122.txt
0.31 In step 2, I would admit to my own fallibility.
The standard answer of “It is the way we have always done it,” should not be acceptable.

-----
0.78 In step 2, I would admit to my own fallibility.
The standard answer of “It is the way we have always done it,

0.30 In step 2, I would admit to my own fallibility.
Though I don't condone gossiping, it is good to know when others are encountering similar situations as you so that you can compare what you have endured and help in coping with the outcomes.

-----
0.67 In step 2, I would admit to my own fallibility.
I told them my story and encouraged them to talk to each other so that they could see that they were not alone in their scenarios.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_1.txt
0.30 In step 2, I would admit to my own fallibility.
I follow up my opinion with the statement of “what do you think of this idea”.

-----
0.80 In step 2, I would admit to my own fallibility.
Whenever I am in a situation that people need to speak up, I try to create a comfortable environment for everyone.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_106.txt
0.31 In step 2, I would admit to my own fallibility.
It has taken me nearly half of my career to step back and observe how others have manage

...

0.40 If someone makes an error, they should not be shamed for this error.
Having the information on hand with actual facts gives empowerment to alleviate fear, anxiety on addressing any subject.

-----
0.73 If someone makes an error, they should not be shamed for this error.
Thats a big problem because misunderstandings can create mistakes.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_14.txt
0.41 If someone makes an error, they should not be shamed for this error.
Our managers are also a human being, and they can also make the wrong decisions.

-----
0.75 If someone makes an error, they should not be shamed for this error.
I used to be scared to speak up on any problem that I found during my work.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_84.txt
0.37 If someone makes an error, they should not be shamed for this error.
Once someone notify me the skin issue, I would be there as fast as possible to do assessment and initiate treatment.

-----
0.69 If someone makes an error,

0.53 If someone makes an error, they should not be shamed for this error.
As mentioned in the video, it was impressed upon me at a very early age that you speak when spoken too, and you do not question authority or your elders because they know better than you.

-----
0.82 If someone makes an error, they should not be shamed for this error.
However, I am the one who will speak up on behalf of someone I feel is being overlooked, disregarded, and ignored.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_49.txt
0.52 If someone makes an error, they should not be shamed for this error.
The entire staff value me as a beneficial aspect to the unit, and because of this, I feel very safe to speak up and ask for assistance when it is required.

-----
0.86 If someone makes an error, they should not be shamed for this error.
I believe the best ways to reduce anxiety of other people why may feel unsafe to speak up is by creating an environment of equality.

---./../../../PhraseModel/StudentEssay/

0.11 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
Reducing anxiety in the work place is important for favorable outcomes for our patients.

-----
0.56 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
Currently, I reduce anxiety with my staff by communication.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_148.txt
0.07 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
I have used my own situations that I have been in to encourage new or timid nurses to always speak up if they are concerned in any way about patient safety.

-----
0.49 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
How do you reduce anxiety of other people who may feel unsafe to speak up?

---./../../../PhraseModel/StudentEssay/M2V3/4gram_141.txt
0.09 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
The culture of the environment of one

0.15 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
I encourage nurses, especially the new ones to ask questions or ask for help when they need it.

-----
0.75 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
I encourage nurses, especially the new ones to ask questions or ask for help when they need it.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_102.txt
0.09 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
How do you reduce anxiety of other people who may feel unsafe to speak up?

-----
0.49 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
How do you reduce anxiety of other people who may feel unsafe to speak up?

---./../../../PhraseModel/StudentEssay/M2V3/4gram_47.txt
0.05 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
The person that is having anxiety needs to feel safe in the environment befor

0.11 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
I tell them that it is their patient as well, and we need to work together to provide good patient care.

-----
0.72 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
When a nurse is anxious to call a doctor at night.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_12.txt
0.11 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
Sharing ideas and concerns will help us all reduce anxiety.

-----
0.66 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
As I reflect on anxiety and psychological factors, I think about my unit and the associates I work with.

---./../../../PhraseModel/StudentEssay/M2V3/4gram_80.txt
0.06 Sometimes all you need to reduce anxiety is to show interest in caring for a person.
By providing support and ensuring them they are not alone may be enough to ease that person's anxie

## Sentence Encoder

In [ ]:
#import tensorflow as tf
import tensorflow_hub as hub
#import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from scipy.spatial import distance
#import seaborn as sns
#import nltk
#nltk.download('punkt')

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/4","https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

## Sentence similarity by Universal sentence encoder

In [14]:
# Similarity by cosine distance
for sent in candidate_sentences:
    candidate_embedding = embed([sent])   
    most_similar_sentences = []
    
    for i in range(11,21): #20 essay files from 1 to 20  
        sentences = get_sentence_array(str(i)+'.txt', sent)
        if len(sentences) == 0:
            continue
        #print sentences
        sent_embeddings = embed(sentences)
        sent_embeddings = np.array(sent_embeddings)
        dists = distance.cdist(candidate_embedding[:1], sent_embeddings, 'cosine')[0, :]
        tuples = [tup for tup in sorted(enumerate(dists), key=lambda x:x[1])]
        #dists = sorted(dists)
        for j in range(0,min(10, len(tuples))):
            most_similar_sentences.append((tuples[j][1], str(i)+'.txt', sentences[tuples[j][0]]))
    most_similar_sentences = sorted(most_similar_sentences, key=lambda x:x[0])
    
    print 'candidate sentence/phrase: ', sent
    print 'Most similar n-grams:'
    for i in range(0,min(7, len(most_similar_sentences)) ):
        if most_similar_sentences[i][0] <= 0.70:
            print most_similar_sentences[i][1], "{:.2f}".format(most_similar_sentences[i][0]), most_similar_sentences[i][2]
        else:
            break
    print '\n'   

candidate sentence/phrase:  everyone's voice and input is valuable and needed.
Most similar n-grams:


candidate sentence/phrase:  feel that they are just as important as everybody around
Most similar n-grams:
13.txt 0.63  I am just naturally concerned with how everyone around me feels—I want everyone around me to feel welcome and comfortable.
17.txt 0.64 We live in a culture that discourages people to speak up and we make people feel unimportant.
12.txt 0.67 By doing these it will help them feel more comfortable to at least speak out with you.
19.txt 0.69  I try to always encourage my co-workers to voice their opinions and speak their minds while in confidence with each other.
12.txt 0.70  Sometimes all you need to reduce anxiety is to show interest in caring for a person.


candidate sentence/phrase:  open up daily conversations.
Most similar n-grams:
14.txt 0.60 After actively listening, I like to share my own stories.
11.txt 0.63 Do they view me as approachable and open to discussi

candidate sentence/phrase:  creating an inviting environment
Most similar n-grams:


candidate sentence/phrase:  not be shamed
Most similar n-grams:


candidate sentence/phrase:  start a conversation and engage them
Most similar n-grams:
12.txt 0.55 Encourage them in discussions or involve them in things at the workplace.
12.txt 0.61 Establish a relationship with that person.
12.txt 0.61 Acknowledging them having conversations or even just greeting them making them feel known.
12.txt 0.63 By doing these it will help them feel more comfortable to at least speak out with you.
14.txt 0.64 So, conversing and allowing people to talk makes a big difference.
19.txt 0.65  I try to always encourage my co-workers to voice their opinions and speak their minds while in confidence with each other.
14.txt 0.66  I like to simply talk and communicate with individuals to try to understand how they feel.


candidate sentence/phrase:  people become comfortable to speak up when they feel heard and invited

# word similarity by cosine metric 

In [68]:
# Similarity by cosine distance
for i in range(1,22): #21 essay files from 1 to 21
    _cache = LRUCache(cache_capacity)
    print '------Essay: ' + str(i)+ '.txt--------\n'
    for word in candidate_words:
            res = recommend_words_by_avg_dist(word,str(i)+'.txt', dist_type='cosine')
            print word, res
            print '\n'
    print '\n\n'

------Essay: 1.txt--------

curiosity [('curiosity', '0.00'), ('work', '0.43'), ('valuable', '0.49'), ('feel', '0.50'), ('seen', '0.51'), ('creating', '0.51'), ('mistake', '0.59'), ('learning', '0.59'), ('anxiety', '0.62'), ('people', '0.62'), ('question', '0.62')]


learning [('learning', '0.00'), ('creating', '0.42'), ('encourage', '0.43'), ('work', '0.49'), ('anxiety', '0.51'), ('workplace', '0.52'), ('patient', '0.53'), ('build', '0.53'), ('process', '0.55'), ('people', '0.56'), ('input', '0.56')]


root_cause [('weakness', '0.35'), ('reduce', '0.38'), ('uncertainty', '0.38'), ('anxiety', '0.39'), ('result', '0.40'), ('reducing', '0.41'), ('process', '0.41'), ('patient', '0.47'), ('psychologically', '0.48'), ('reduces', '0.49'), ('needed', '0.49')]


encourage [('encourage', '0.00'), ('contribute', '0.27'), ('creating', '0.31'), ('build', '0.34'), ('reduce', '0.41'), ('safe', '0.43'), ('learning', '0.43'), ('needed', '0.45'), ('allows', '0.45'), ('workplace', '0.46'), ('safer', '0.

environment [('need', '0.30'), ('information', '0.42'), ('understand', '0.44'), ('practice', '0.47'), ('reduce', '0.48'), ('experience', '0.48'), ('encourage', '0.49'), ('lack', '0.50'), ('help', '0.51'), ('way', '0.51'), ('people', '0.51')]


inviting [('asking', '0.29'), ('conversation', '0.36'), ('talk', '0.40'), ('engage', '0.41'), ('gathered', '0.41'), ('wanted', '0.46'), ('speak', '0.46'), ('want', '0.49'), ('encourage', '0.50'), ('question', '0.50'), ('help', '0.52')]


participation [('encourage', '0.39'), ('engage', '0.39'), ('country', '0.42'), ('experience', '0.47'), ('motivate', '0.49'), ('people', '0.51'), ('practice', '0.51'), ('need', '0.53'), ('making', '0.54'), ('lack', '0.56'), ('question', '0.57')]


confidence [('question', '0.39'), ('way', '0.44'), ('feel', '0.46'), ('want', '0.46'), ('lack', '0.47'), ('need', '0.48'), ('help', '0.49'), ('wanted', '0.49'), ('anybody', '0.50'), ('asking', '0.50'), ('heshe', '0.52')]


confidencebuilding [('country', '0.52'), ('reduc

participation [('leadership', '0.44'), ('creating', '0.51'), ('people', '0.51'), ('promotes', '0.51'), ('improve', '0.55'), ('inviting', '0.56'), ('culture', '0.56'), ('question', '0.57'), ('learning', '0.59'), ('invited', '0.59'), ('environment', '0.60')]


confidence [('question', '0.39'), ('mistake', '0.45'), ('feel', '0.46'), ('empowered', '0.49'), ('safe', '0.49'), ('help', '0.49'), ('asking', '0.50'), ('comfortable', '0.52'), ('leadership', '0.53'), ('improve', '0.54'), ('people', '0.54')]


confidencebuilding [('reduce', '0.55'), ('improve', '0.60'), ('question', '0.60'), ('environment', '0.63'), ('empowered', '0.64'), ('safe', '0.67'), ('quality', '0.68'), ('safety', '0.68'), ('people', '0.72'), ('anxiety', '0.72'), ('patient', '0.72')]


orientation [('question', '0.50'), ('creating', '0.53'), ('learning', '0.55'), ('environment', '0.56'), ('culture', '0.58'), ('conversation', '0.58'), ('psychological', '0.58'), ('important', '0.61'), ('quality', '0.62'), ('leadership', '0.62'

guilty [('mistake', '0.39'), ('person', '0.44'), ('admit', '0.50'), ('werent', '0.51'), ('willing', '0.52'), ('fact', '0.52'), ('everyones', '0.52'), ('quo', '0.54'), ('question', '0.55'), ('afraid', '0.55'), ('ask', '0.55')]


leadership [('accountability', '0.43'), ('teamwork', '0.44'), ('ideology', '0.47'), ('member', '0.47'), ('excellence', '0.49'), ('workplace', '0.51'), ('encouraging', '0.53'), ('learning', '0.54'), ('communication', '0.54'), ('critique', '0.55'), ('idea', '0.56')]


support [('encouraging', '0.31'), ('accountability', '0.37'), ('creating', '0.41'), ('communication', '0.43'), ('teamwork', '0.44'), ('offer', '0.44'), ('better', '0.46'), ('discourages', '0.48'), ('workplace', '0.49'), ('learning', '0.49'), ('safety', '0.50')]


rounding [('reduced', '0.46'), ('input', '0.48'), ('fewer', '0.49'), ('mistake', '0.54'), ('making', '0.55'), ('draw', '0.55'), ('difficult', '0.55'), ('result', '0.56'), ('caused', '0.59'), ('weakness', '0.59'), ('framing', '0.60')]


visib

daily_basis [('people', '0.45'), ('safe', '0.48'), ('need', '0.48'), ('workplace', '0.49'), ('patient', '0.52'), ('time', '0.52'), ('better', '0.57'), ('sure', '0.57'), ('unsafe', '0.57'), ('able', '0.61'), ('outcome', '0.64')]


conversation [('speak', '0.35'), ('discussing', '0.39'), ('talked', '0.42'), ('silence', '0.44'), ('know', '0.45'), ('feel', '0.50'), ('people', '0.51'), ('sure', '0.52'), ('fact', '0.53'), ('instead', '0.57'), ('encouraged', '0.59')]


conversational [('speak', '0.39'), ('feel', '0.49'), ('instead', '0.52'), ('discussing', '0.52'), ('know', '0.52'), ('better', '0.56'), ('outcome', '0.57'), ('sure', '0.58'), ('fact', '0.58'), ('need', '0.58'), ('error', '0.59')]


open [('rest', '0.50'), ('safe', '0.58'), ('time', '0.60'), ('able', '0.61'), ('workplace', '0.64'), ('better', '0.66'), ('right', '0.68'), ('need', '0.68'), ('instead', '0.68'), ('looked', '0.69'), ('team', '0.70')]


communication [('right', '0.58'), ('need', '0.61'), ('workplace', '0.64'), ('conce

...

accountability [('safety', '0.41'), ('reduce', '0.47'), ('power', '0.55'), ('department', '0.55'), ('staff', '0.56'), ('question', '0.57'), ('importance', '0.58'), ('teamwork', '0.59'), ('allows', '0.61'), ('emphasis', '0.61'), ('psychological', '0.65')]


insecurity [('anxiety', '0.28'), ('afraid', '0.41'), ('reassuring', '0.48'), ('question', '0.50'), ('psychological', '0.52'), ('way', '0.53'), ('importance', '0.53'), ('genuinely', '0.54'), ('tolerate', '0.54'), ('reduce', '0.58'), ('emphasis', '0.60')]


speak [('speak', '0.00'), ('speaking', '0.12'), ('ask', '0.32'), ('communicate', '0.34'), ('asking', '0.37'), ('genuinely', '0.40'), ('afraid', '0.41'), ('talk', '0.41'), ('ridiculing', '0.48'), ('question', '0.48'), ('way', '0.53')]


provider [('allows', '0.58'), ('communicate', '0.61'), ('monthly', '0.62'), ('staff', '0.63'), ('safety', '0.64'), ('group', '0.67'), ('department', '0.68'), ('power', '0.71'), ('reduce', '0.72'), ('open', '0.73'), ('treating', '0.77')]


anxiety [('a

relatable [('empathy', '0.51'), ('helpful', '0.51'), ('honest', '0.53'), ('anxious', '0.56'), ('experience', '0.57'), ('speak', '0.57'), ('learn', '0.58'), ('whats', '0.59'), ('anxiety', '0.65'), ('best', '0.67'), ('way', '0.67')]


personal [('experience', '0.28'), ('honest', '0.31'), ('helpful', '0.34'), ('practice', '0.35'), ('empathy', '0.36'), ('question', '0.38'), ('anxious', '0.45'), ('support', '0.46'), ('opinion', '0.46'), ('necessary', '0.47'), ('anxiety', '0.47')]


relationship [('experience', '0.33'), ('empathy', '0.38'), ('problem', '0.40'), ('question', '0.43'), ('anxiety', '0.45'), ('way', '0.45'), ('practice', '0.46'), ('anxious', '0.46'), ('honest', '0.47'), ('helpful', '0.49'), ('lead', '0.51')]


policy [('effort', '0.34'), ('practice', '0.34'), ('education', '0.38'), ('support', '0.39'), ('question', '0.43'), ('necessary', '0.45'), ('reduce', '0.45'), ('encourage', '0.46'), ('opinion', '0.46'), ('problem', '0.47'), ('communication', '0.49')]


demonstrate [('encour

inviting [('asking', '0.29'), ('ask', '0.32'), ('conversation', '0.36'), ('hear', '0.44'), ('question', '0.50'), ('help', '0.52'), ('inperson', '0.52'), ('address', '0.52'), ('free', '0.53'), ('email', '0.53'), ('read', '0.54')]


participation [('workplace', '0.45'), ('responsibility', '0.50'), ('people', '0.51'), ('issue', '0.51'), ('stronger', '0.54'), ('inperson', '0.54'), ('making', '0.54'), ('subject', '0.55'), ('free', '0.55'), ('question', '0.57'), ('address', '0.58')]


confidence [('question', '0.39'), ('way', '0.44'), ('anxious', '0.45'), ('stronger', '0.45'), ('feel', '0.46'), ('person', '0.46'), ('emotion', '0.46'), ('responsibility', '0.47'), ('clear', '0.48'), ('wrong', '0.49'), ('help', '0.49')]


confidencebuilding [('clarification', '0.58'), ('responsibility', '0.58'), ('decrease', '0.60'), ('question', '0.60'), ('address', '0.62'), ('stronger', '0.63'), ('issue', '0.65'), ('problem', '0.67'), ('difficult', '0.68'), ('clarify', '0.68'), ('overall', '0.70')]


orientat

positive [('negative', '0.17'), ('attitude', '0.27'), ('openness', '0.27'), ('constructive', '0.29'), ('criticism', '0.33'), ('anxiety', '0.34'), ('ability', '0.37'), ('mind', '0.43'), ('confidence', '0.43'), ('encouraging', '0.44'), ('enhance', '0.44')]


positive_feedback [('negative', '0.23'), ('ability', '0.41'), ('openness', '0.43'), ('anxiety', '0.45'), ('constructive', '0.46'), ('decrease', '0.47'), ('criticism', '0.49'), ('lead', '0.52'), ('mind', '0.52'), ('attitude', '0.52'), ('enhance', '0.57')]


positive_response [('criticism', '0.44'), ('negative', '0.47'), ('confidence', '0.65'), ('opinion', '0.65'), ('absolutely', '0.66'), ('openness', '0.66'), ('imaginative', '0.69'), ('improvement', '0.71'), ('increased', '0.71'), ('mind', '0.72'), ('anxiety', '0.72')]


environment [('environment', '0.00'), ('process', '0.38'), ('ability', '0.39'), ('enhance', '0.43'), ('openness', '0.46'), ('practice', '0.47'), ('improvement', '0.48'), ('encourage', '0.49'), ('protect', '0.50'), ('c

insecurity [('anxiety', '0.28'), ('feeling', '0.30'), ('feel', '0.43'), ('experience', '0.46'), ('need', '0.47'), ('question', '0.50'), ('result', '0.53'), ('empowering', '0.56'), ('workplace', '0.56'), ('health', '0.56'), ('amazed', '0.57')]


speak [('speak', '0.00'), ('know', '0.29'), ('ask', '0.32'), ('feel', '0.42'), ('feeling', '0.43'), ('question', '0.48'), ('peer', '0.50'), ('experience', '0.53'), ('suggestion', '0.54'), ('freely', '0.54'), ('need', '0.56')]


provider [('healthcare', '0.26'), ('communication', '0.31'), ('delivery', '0.37'), ('medicare', '0.44'), ('health', '0.48'), ('support', '0.48'), ('care', '0.51'), ('non', '0.57'), ('need', '0.58'), ('workplace', '0.61'), ('individual', '0.62')]


anxiety [('anxiety', '0.00'), ('feeling', '0.20'), ('experience', '0.33'), ('feel', '0.38'), ('question', '0.42'), ('approach', '0.46'), ('need', '0.49'), ('insight', '0.51'), ('result', '0.52'), ('amazed', '0.52'), ('workplace', '0.53')]


gain [('gain', '0.00'), ('gaining', '0

In [71]:
# all essay's combined
for i in range(0,1): # 0.txt contains all essays 
    _cache = LRUCache(cache_capacity)
    print '------Essay: ' + str(i)+ '.txt--------\n'
    for word in candidate_words:
            res = recommend_words_by_avg_dist(word,str(i)+'.txt', dist_type='cosine')
            print word, res
            print '\n'
    print '\n\n'

------Essay: 0.txt--------

curiosity [('curiosity', '0.00'), ('imaginative', '0.30'), ('look', '0.31'), ('curious', '0.31'), ('incredibly', '0.35'), ('mind', '0.38'), ('insight', '0.38'), ('thing', '0.39'), ('find', '0.39'), ('everyday', '0.39')]


learning [('learning', '0.00'), ('education', '0.25'), ('curriculum', '0.28'), ('experience', '0.30'), ('learn', '0.30'), ('insight', '0.32'), ('onetoone', '0.34'), ('teamwork', '0.34'), ('motivation', '0.36'), ('motivate', '0.36')]


root_cause [('problem', '0.20'), ('correcting', '0.31'), ('situation', '0.35'), ('weakness', '0.35'), ('outcome', '0.36'), ('error', '0.36'), ('response', '0.37'), ('cause', '0.37'), ('potential', '0.38'), ('reduce', '0.38')]


encourage [('encourage', '0.00'), ('encouraging', '0.07'), ('enhance', '0.19'), ('motivate', '0.19'), ('encouraged', '0.19'), ('improve', '0.20'), ('bring', '0.20'), ('engage', '0.23'), ('promotes', '0.24'), ('opportunity', '0.25')]


encouraging [('encouraging', '0.00'), ('encourage', 

retaliation [('retaliation', '0.00'), ('blame', '0.29'), ('incident', '0.30'), ('punished', '0.32'), ('humiliated', '0.34'), ('offend', '0.38'), ('abusing', '0.38'), ('hid', '0.40'), ('apologized', '0.41'), ('incompetent', '0.41')]


role_model [('caring', '0.35'), ('empowering', '0.37'), ('motivate', '0.39'), ('knowledgeable', '0.40'), ('socially', '0.41'), ('empathy', '0.42'), ('confident', '0.43'), ('openly', '0.43'), ('honest', '0.45'), ('advocate', '0.45')]


demonstrated [('utilized', '0.40'), ('effective', '0.42'), ('correcting', '0.43'), ('develop', '0.44'), ('conducted', '0.45'), ('seen', '0.45'), ('discussed', '0.45'), ('described', '0.46'), ('contribute', '0.46'), ('potential', '0.46')]


leader [('leadership', '0.18'), ('member', '0.33'), ('party', '0.39'), ('advocate', '0.41'), ('fellow', '0.41'), ('leading', '0.44'), ('disciplined', '0.47'), ('supported', '0.47'), ('meeting', '0.48'), ('ideology', '0.49')]


guilt [('feeling', '0.22'), ('silence', '0.27'), ('wrong', '0.28

# word similarity by euclidean metric

In [ ]:
# Similarity by Euclidean distance
for i in range(1,22): #21 essay files from 1 to 21
    _cache = LRUCache(cache_capacity)
    print '------Essay: ' + str(i)+ '.txt--------\n'
    for word in candidate_words:
            res = recommend_words_by_avg_dist(word,str(i)+'.txt') #dist_type='euclidean'
            print word, res
            print '\n'
    print '\n\n'